In [16]:
import sys
from pathlib import Path
from datetime import datetime

# Añade src al path para importar los módulos
sys.path.append(str(Path().resolve().parent / 'src'))

import config

In [17]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
import hopsworks

# Conexión a hopsworks y proyecto   
project = hopsworks.login(project=config.HOPSWORKS_PROJECT_NAME, api_key_value=config.HOPSWORKS_API_KEY)
feature_store = project.get_feature_store()


print("Conectado a proyecto:", project.name)
print("Conectado al feature store:", feature_store.name)


2025-08-23 18:51:28,551 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-08-23 18:51:28,554 INFO: Initializing external client
2025-08-23 18:51:28,555 INFO: Base URL: https://c.app.hopsworks.ai:443
Connection closed.
2025-08-23 18:51:28,554 INFO: Initializing external client
2025-08-23 18:51:28,555 INFO: Base URL: https://c.app.hopsworks.ai:443




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'


2025-08-23 18:51:29,744 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1242272
Conectado a proyecto: fleca_mlops
Conectado al feature store: fleca_mlops_featurestore

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1242272
Conectado a proyecto: fleca_mlops
Conectado al feature store: fleca_mlops_featurestore


In [19]:
from datetime import timedelta
import pandas as pd

FG_NAME = "times_series_bolleria_feature_group"
FV_NAME = "times_series_bolleria_feature_view"
FV_VERSION = 1

# Lectura batch cruda de feature views
feature_views= feature_store.get_feature_view(name=FV_NAME, version=FV_VERSION)
ts_df = feature_views.get_batch_data(
    start_date=None,
    end_date=None,
    features=None,
    )

# Reseteo de índice y ordenación por la columna de fecha antes de visualizar
ts_df = ts_df.sort_values('week_start').reset_index(drop=True)
ts_df.head(5)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.70s) 
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.70s) 


,familia,base_imponible,is_summer_peak,is_easter,week_start
0,BOLLERIA,825.11,0,0,2023-01-02 00:00:00+00:00
1,BOLLERIA,658.40,0,0,2023-01-09 00:00:00+00:00
2,BOLLERIA,741.40,0,0,2023-01-16 00:00:00+00:00
3,BOLLERIA,653.64,0,0,2023-01-23 00:00:00+00:00
4,BOLLERIA,680.46,0,0,2023-01-30 00:00:00+00:00


In [20]:
from src.model import transformar_features_target

# Procesar datos usando la función mejorada que acepta tuplas directamente
ts_df = transformar_features_target(
    ts_df,
    lags_list=[1, 2, 3, 52], 
    columna_target='base_imponible',
    cols_exogenas=['is_easter', 'is_summer_peak'],
    periodos_adelante=1,
    eliminar_nulos=True,
    return_format='dataframe'  # Obtenemos un único DataFrame con features y target
)

# Resetear el índice
ts_df = ts_df.reset_index(drop=True)

# Mostrar las primeras filas del DataFrame transformado
ts_df.head(5)

2025-08-23 18:51:36,256 INFO: Usando DataFrame de entrada: (133, 5)
2025-08-23 18:51:36,260 INFO: Retornando DataFrame combinado: (80, 8)
2025-08-23 18:51:36,260 INFO: Retornando DataFrame combinado: (80, 8)


,base_imponible_lag1,base_imponible_lag2,base_imponible_lag3,base_imponible_lag52,is_easter,is_summer_peak,week_start,target
0,572.51,534.79,563.18,825.11,0,0,2024-01-15 00:00:00+00:00,680.30
1,597.65,572.51,534.79,658.40,0,0,2024-01-22 00:00:00+00:00,603.99
2,680.30,597.65,572.51,741.40,0,0,2024-01-29 00:00:00+00:00,600.14
3,603.99,680.30,597.65,653.64,0,0,2024-02-05 00:00:00+00:00,689.32
4,600.14,603.99,680.30,680.46,0,0,2024-02-12 00:00:00+00:00,627.76


In [21]:
# Cargamos el modelo para predecir
import mlflow.pyfunc

MODEL_NAME = 'RandomForest'
MODEL_VERSION = 1  # Debe ser un entero, no un string

model = mlflow.pyfunc.load_model(model_uri=f'models:/{MODEL_NAME}/{MODEL_VERSION}')
print('Modelo cargado:', model)


MlflowException: Registered Model with name=RandomForest not found

In [23]:
# Validar tracking URI y listar modelos registrados en MLflow
import mlflow
from mlflow.tracking import MlflowClient

# Mostrar el tracking URI actual
print('Tracking URI:', mlflow.get_tracking_uri())

# Listar modelos registrados en el Model Registry (compatible con versiones antiguas)
client = MlflowClient()
models = client.search_registered_models()
print('Modelos registrados:')
for m in models:
    print(f"- {m.name} (versiones: {[v.version for v in m.latest_versions]})")

Tracking URI: file:///c:/Workspace/mlops_fleca_project/notebooks/mlruns
Modelos registrados:
